# 텍스트 이진 분류 (긍정 or 부정)

## 0. 코랩 연결 끊김 방지

```
function ClickConnect(){
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End"); 
};
var clickid = setInterval(ClickConnect, 60*1000)
```

```
clearInterval(clickid);
```

## 1. Colab 환경 설정

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

## 2. 라이브러리 설치

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/sentiment-classification

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
  Using cached transformers-3.0.0-py3-none-any.whl (754 kB)
  Using cached tokenizers-0.8.0rc4-cp38-cp38-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.2.3 requires transformers<=4.8.1,>=4.8.1, but yo

  Using cached tokenizers-0.10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.0rc4
    Uninstalling tokenizers-0.8.0rc4:
      Successfully uninstalled tokenizers-0.8.0rc4
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.0
    Uninstalling transformers-3.0.0:
      Successfully uninstalled transformers-3.0.0
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## 3. 라이브러리 준비

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

## 4. 데이터 준비

In [ ]:
!git clone https://github.com/e9t/nsmc.git
!git clone https://github.com/bab2min/corpus.git

In [ ]:
import pandas as pd

naver_movie_train_data = pd.read_table("nsmc/ratings_train.txt")
naver_movie_test_data = pd.read_table("nsmc/ratings_test.txt")
naver_shopping_data = pd.read_table("corpus/sentiment/naver_shopping.txt")
steam_data = pd.read_table("corpus/sentiment/steam.txt")

## 5. 데이터 전처리

In [ ]:
naver_movie_train_data = naver_movie_train_data.drop(['id'], axis=1)
naver_movie_test_data = naver_movie_test_data.drop(['id'], axis=1)

naver_shopping_data.columns = ['label', 'document']
naver_shopping_data.loc[naver_shopping_data['label'] == 1, 'label'] = 0
naver_shopping_data.loc[naver_shopping_data['label'] == 2, 'label'] = 0
naver_shopping_data.loc[naver_shopping_data['label'] == 4, 'label'] = 1
naver_shopping_data.loc[naver_shopping_data['label'] == 5, 'label'] = 1

steam_data.columns = ['label', 'document']

In [ ]:
naver_movie_train_data.drop_duplicates(['document'], inplace=True)
naver_movie_test_data.drop_duplicates(['document'], inplace=True)
naver_shopping_data.drop_duplicates(['document'], inplace=True)
steam_data.drop_duplicates(['document'], inplace=True)

In [ ]:
naver_movie_train_data = naver_movie_train_data.dropna()
naver_movie_test_data = naver_movie_test_data.dropna()

In [ ]:
import numpy as np

# 한글이 아닌 것 지우기
naver_movie_train_data['document'] = naver_movie_train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
naver_movie_test_data['document'] = naver_movie_test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
naver_shopping_data['document'] = naver_shopping_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
steam_data['document'] = steam_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
# 공백 모두 지우기
naver_movie_train_data['document'] = naver_movie_train_data['document'].str.replace('^ +', "", regex=True)
naver_movie_test_data['document'] = naver_movie_test_data['document'].str.replace('^ +', "", regex=True)
naver_shopping_data['document'] = naver_shopping_data['document'].str.replace('^ +', "", regex=True)
steam_data['document'] = steam_data['document'].str.replace('^ +', "", regex=True)
# 빈 값 Nan으로 채우기
naver_movie_train_data['document'].replace('', np.nan, inplace=True)
naver_movie_test_data['document'].replace('', np.nan, inplace=True)
naver_shopping_data['document'].replace('', np.nan, inplace=True)
steam_data['document'].replace('', np.nan, inplace=True)
# Nan값 제거
naver_movie_train_data = naver_movie_train_data.dropna()
naver_movie_test_data = naver_movie_test_data.dropna()
naver_shopping_data = naver_shopping_data.dropna()
steam_data = steam_data.dropna()

## 6. train, test 세트 분리

In [ ]:
from sklearn.model_selection import train_test_split
naver_shopping_train_data, naver_shopping_test_data = train_test_split(naver_shopping_data, test_size=0.25)
steam_train_data, steam_test_data = train_test_split(steam_data, test_size=0.25)

In [ ]:
train_data = pd.concat([naver_movie_train_data, naver_shopping_train_data, steam_train_data])
test_data = pd.concat([naver_movie_test_data, naver_shopping_test_data, steam_test_data])

train_data = train_data.sample(frac=1).reset_index(drop=True) 
test_data = test_data.sample(frac=1).reset_index(drop=True) 

## 7. 데이터 변환

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset.itertuples()]
        self.labels = [np.int32(i[label_idx]) for i in dataset.itertuples()]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
data_train = BERTDataset(train_data, 1, 2, tok, max_len, True, False)
data_test = BERTDataset(test_data, 1, 2, tok, max_len, True, False)

In [ ]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 8. 훈련

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_368/1704445530.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/5786 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7290350794792175 train acc 0.515625
epoch 1 batch id 201 loss 0.6995196342468262 train acc 0.5209888059701493
epoch 1 batch id 401 loss 0.5866668820381165 train acc 0.6052447007481296
epoch 1 batch id 601 loss 0.32097765803337097 train acc 0.6689631863560732
epoch 1 batch id 801 loss 0.41163352131843567 train acc 0.7084698813982522
epoch 1 batch id 1001 loss 0.3782798945903778 train acc 0.7335477022977023
epoch 1 batch id 1201 loss 0.23876965045928955 train acc 0.7523808284762697
epoch 1 batch id 1401 loss 0.2695901393890381 train acc 0.7663610813704497
epoch 1 batch id 1601 loss 0.39511433243751526 train acc 0.7769460493441599
epoch 1 batch id 1801 loss 0.42404916882514954 train acc 0.7856485980011105
epoch 1 batch id 2001 loss 0.42690467834472656 train acc 0.7930019365317341
epoch 1 batch id 2201 loss 0.32362300157546997 train acc 0.7995513402998637
epoch 1 batch id 2401 loss 0.406591534614563 train acc 0.8052178779675135
epoch 1 batch id 2601 loss 0.2732058

/tmp/ipykernel_368/1704445530.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1935 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8903305754323196


  0%|          | 0/5786 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.1908959597349167 train acc 0.9375
epoch 5 batch id 201 loss 0.1060066819190979 train acc 0.9614427860696517
epoch 5 batch id 401 loss 0.0872025117278099 train acc 0.9601387157107232
epoch 5 batch id 601 loss 0.13215786218643188 train acc 0.9594165973377704
epoch 5 batch id 801 loss 0.20559781789779663 train acc 0.9611423220973783
epoch 5 batch id 1001 loss 0.1769813597202301 train acc 0.9608204295704296
epoch 5 batch id 1201 loss 0.010505618527531624 train acc 0.96189373438801
epoch 5 batch id 1401 loss 0.0377800352871418 train acc 0.9625156138472519
epoch 5 batch id 1601 loss 0.05036428943276405 train acc 0.9629918800749532
epoch 5 batch id 1801 loss 0.057756535708904266 train acc 0.9635705857856747
epoch 5 batch id 2001 loss 0.09551557153463364 train acc 0.9636509870064968
epoch 5 batch id 2201 loss 0.008611119352281094 train acc 0.9640291344843253
epoch 5 batch id 2401 loss 0.30358678102493286 train acc 0.9644614223240316
epoch 5 batch id 2601 loss 0.099910

  0%|          | 0/1935 [00:00<?, ?it/s]

epoch 5 test acc 0.8918163635460148


## 9. 테스트

In [ ]:
# 테스트 문장 예측
test_sentence = '무난하네요.'
test_label = 1 # 실제 정답

In [ ]:
unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['document', 'label']])
# unseen_values = unseen_test.values
test_set = BERTDataset(unseen_test, 1, 2, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    print(out)

/tmp/ipykernel_368/264004550.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[-0.1702,  0.1392]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
result = torch.nn.functional.softmax(out)
result

/tmp/ipykernel_368/951339506.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  result = torch.nn.functional.softmax(out)


tensor([[0.4233, 0.5767]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

## 10. 모델 저장 및 불러오기

In [ ]:
torch.save(model.state_dict(), "./best_weights_v2.pt")
torch.save(model, "./best_v2.pt")

In [ ]:
loaded_model = torch.load('best_v2.pt')

In [ ]:
# 테스트 문장 예측
test_sentence = '너무 별로에요, 사지마세요. 돈낭비입니다.'
test_label = 0 # 실제 정답

In [ ]:
unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['document', 'label']])
# unseen_values = unseen_test.values
test_set = BERTDataset(unseen_test, 1, 2, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = loaded_model(token_ids, valid_length, segment_ids)
    print(out)

/tmp/ipykernel_368/809299586.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[ 3.7407, -3.1388]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
result = torch.nn.functional.softmax(out)
result

/tmp/ipykernel_368/951339506.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  result = torch.nn.functional.softmax(out)


tensor([[0.9990, 0.0010]], device='cuda:0', grad_fn=<SoftmaxBackward0>)